In [1]:
import pandas as pd, numpy as np

In [2]:
klstVedurPath = 'D:\Skóli\lokaverkefni_vel\data\Vedurstofa\stripped_klst.feather'
mergedPath = 'D:\Skóli\lokaverkefni_vel\data\merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather'

In [3]:
df_vedur_klst = pd.read_feather(klstVedurPath)
df_merged = pd.read_feather(mergedPath)

In [4]:
df_vedur_klst.timi = pd.to_datetime(df_vedur_klst.timi)
df_merged.time = pd.to_datetime(df_merged.time)

In [5]:
# Rounds to next hour so as to be able to directly compare with the vedur klst file
def next_hour(time):
    return time.ceil('h')

In [6]:
df_merged['timi'] = df_merged.time.apply(next_hour)

df_merged

,X,Y,time,ws_15,ws_250,ws_500,wd_15,wd_250,wd_500,p_15,...,longitude,latitude,Ri_01,Ri_12,N_01,N_12,station_elevation,landscape_points,elevations,timi
0,419760.4353,311410.4631,2020-02-14 09:40:00,19.6041,23.1354,23.5411,97.9485,99.3434,99.9549,98956.6580,...,-20.5995,63.2993,-0.593939,-1.137324,-0.000134,-0.000003,33.438521,"[[420112.93989447743, 311361.54904540273], [42...","[779.7168035654618, 782.0526596557524, 786.179...",2020-02-14 10:00:00
1,419760.4353,311410.4631,2019-02-20 01:00:00,14.4611,16.8599,17.1642,145.3825,146.3173,146.5913,100351.7109,...,-20.5995,63.2993,-0.422328,-1.799695,-0.000044,-0.000003,33.438521,"[[420112.4996219488, 311358.474764043], [42011...","[777.3745944169202, 782.0397823265109, 786.175...",2019-02-20 01:00:00
2,419760.4353,311410.4631,2019-02-17 00:40:00,8.5471,9.1812,9.1686,62.6103,65.3626,79.3139,100649.9705,...,-20.5995,63.2993,-1.177507,26685.778689,-0.000009,0.000068,33.438521,"[[420116.31686465454, 311409.84196886426], [42...","[784.1650077215852, 788.0848430684046, 790.183...",2019-02-17 01:00:00
3,419760.4353,311410.4631,2019-02-05 23:40:00,10.3157,11.1665,11.1894,88.1111,89.3584,90.2917,101001.3064,...,-20.5995,63.2993,-2.864259,233.822195,-0.000038,0.000002,33.438521,"[[420114.5577725153, 311375.11697822495], [420...","[779.7686245550574, 784.150857915831, 786.2225...",2019-02-06 00:00:00
4,419760.4353,311410.4631,2018-12-17 19:30:00,11.2918,17.3783,17.6068,116.0218,129.5856,136.2121,99682.9219,...,-20.5995,63.2993,0.412797,245.814617,0.000277,0.000205,33.438521,"[[420116.29789345217, 311406.73637940787], [42...","[784.1530257740433, 786.2250760674577, 790.170...",2018-12-17 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18428,543704.3472,672486.1026,2014-12-10 12:40:00,10.6554,11.5584,11.0643,6.4581,8.6411,12.4018,96755.1484,...,-18.0167,66.5438,-7.538465,4.194781,-0.000111,0.000016,19.654911,"[[543906.9435960962, 672778.6889533278], [5438...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2014-12-10 13:00:00
18429,543704.3472,672486.1026,2015-12-08 05:00:00,25.4698,31.2753,33.5225,97.3362,98.8644,101.6907,99002.3281,...,-18.0167,66.5438,-0.042580,1.078334,-0.000026,0.000087,19.654911,"[[544054.4942800443, 672422.4700910029], [5440...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2015-12-08 05:00:00
18430,543704.3472,672486.1026,2019-12-11 08:10:00,19.3720,22.8016,23.2493,41.1882,44.0706,46.2614,95774.5937,...,-18.0167,66.5438,-0.110178,12.661773,-0.000023,0.000041,19.654911,"[[543800.6490567861, 672828.7073668151], [5437...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2019-12-11 09:00:00
18431,543704.3472,672486.1026,2020-02-14 14:10:00,23.5278,28.6951,29.9320,93.5311,95.0293,96.2047,97827.6385,...,-18.0167,66.5438,-0.264139,1.104732,-0.000128,0.000027,19.654911,"[[544055.1414271548, 672426.1402446448], [5440...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-02-14 15:00:00


In [7]:
df_merged.columns

Index(['X', 'Y', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250',
       'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'stod',
       'f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'Ri_01',
       'Ri_12', 'N_01', 'N_12', 'station_elevation', 'landscape_points',
       'elevations', 'timi'],
      dtype='object')

In [8]:
df_vedur_klst.columns

Index(['timi', 'stod', 'f', 'fx', 'fg', 'd', 'dsdev'], dtype='object')

In [18]:
df_vedur_klst.stod = df_vedur_klst.stod.astype(np.int64)
df_vedur_klst.fg = pd.to_numeric(df_vedur_klst.fg, errors = 'coerce', downcast='float')

In [36]:
df_final = pd.merge(df_merged, df_vedur_klst, on = ['stod', 'timi'], how = 'inner')

In [37]:
len(df_final)

1183